In [ ]:
import sys, os
print (sys.version)
import cv2, math
import time
from matplotlib import pyplot as plt
import signal
import numpy as np


print(cv2.__file__)
%matplotlib notebook

# Import and Setup OpenPose

In [ ]:
openPoseDir = "/home/yousof/AI/openpose/"
openPoseLib = openPoseDir + "build/python"
sys.path.append(openPoseLib);
from openpose import pyopenpose as op

params = dict()
params["model_folder"] = openPoseDir + "models"
params["hand"] = True
params["hand_detector"] = 0

In [ ]:
def plotOpenCVImage(img):
    plt.figure(figsize=(10,18))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

# Initialize OpenPose

In [ ]:
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()
datum = op.Datum()

In [ ]:
def wait_until(condition, interval=0.1, timeout=1, *args):
    start = time.time()
    while not condition(*args) and time.time() - start < timeout:
        time.sleep(interval)

def pointsToDegrees(datum):
    BaseIdx = 5
    midIdx = 6
    tipIdx = 8
    hand = 0
    if datum.handKeypoints[hand].size != 1 :
        points = datum.handKeypoints[hand][0,:,:].squeeze()
        all_zeros = not np.any(points)
        if not all_zeros:
            points[:,1] = datum.cvOutputData.shape[1] - points[:,1]
            dy = points[midIdx, 1] - points[BaseIdx, 1]
            dx = points[midIdx, 0] - points[BaseIdx, 0]
            deg1 = math.degrees(math.atan2(dy, dx))
            dy = points[tipIdx, 1] - points[midIdx, 1]
            dx = points[tipIdx, 0] - points[midIdx, 0]
            deg2 = math.degrees(math.atan2(dy, dx))
            print(deg1-90, deg2)
    
def processImage(img):
    global datum, opWrapper
    datum.cvInputData = img
    opWrapper.waitAndEmplace([datum])
    if opWrapper.waitAndPop([datum]):
        pointsToDegrees(datum)

# Read from webcam

In [ ]:
cam = cv2.VideoCapture(0)
cam.set(3 , 1920  ) # width        
cam.set(4 , 1080  ) # height       
#cam.set(10, 120  ) # brightness     min: 0   , max: 255 , increment:1  
#cam.set(11, 50   ) # contrast       min: 0   , max: 255 , increment:1     
#cam.set(12, 70   ) # saturation     min: 0   , max: 255 , increment:1
#cam.set(13, 13   ) # hue         
#cam.set(14, 50   ) # gain           min: 0   , max: 127 , increment:1
#cam.set(15, -3   ) # exposure       min: -7  , max: -1  , increment:1
#cam.set(17, 5000 ) # white_balance  min: 4000, max: 7000, increment:1
#cam.set(28, 0    ) # focus          min: 0   , max: 255 , increment:5

In [ ]:
is_capturing, frame = cam.read()
cam.release()

In [ ]:
plotOpenCVImage(frame)
processImage(frame)
plotOpenCVImage(datum.cvOutputData)


In [ ]:
datum.handKeypoints

In [ ]:
datum.handKeypoints[1].size